In [1]:
import numpy as np
from fastl2lir import FastL2LiR
import os
import pandas as pd
from itertools import permutations

from fmri_reconstruction_with_dmvae.deeprecon.datasets.load import load_data

In [2]:
data_path = "/home/acg17270jl/projects/fmri-reconstruction-with-dmvae/data/deeprecon/"
conversion_output_path = "/home/acg17270jl/projects/fmri-reconstruction-with-dmvae/results/deeprecon/conversion/"
subj_list = [1, 2, 3, 4, 5]  # [1, 2, 3, 4, 5]

num_samples = 2400

# set needed repetitions
rep = 1 if num_samples < 1200 else int(num_samples / 1200)

In [3]:
### Load data
train_brain_data_dict, test_brain_data_dict, all_subj_num_voxels = load_data(data_path, subj_list)

### Load model params from "https://github.com/KamitaniLab/InterIndividualDeepImageReconstruction/blob/main/ridge_ncc/params/converter_params.csv"
csv_path = "/home/acg17270jl/projects/fmri-reconstruction-with-dmvae/checkpoints/deeprecon/converter_regression/converter_params.csv"

converter_params_df = pd.read_csv(csv_path).query("Method == 'wholeVC'")

converter_params_df = converter_params_df.assign(
    Source=lambda x: x["Source"].str.extract(r"(\d+)", expand=False),
    Target=lambda x: x["Target"].str.extract(r"(\d+)", expand=False),
)

target_num = converter_params_df["Target"].astype(int)
source_num = converter_params_df["Source"].astype(int)
order = np.lexsort([source_num, target_num])
converter_params_df = converter_params_df.iloc[order]

converter_params_df = converter_params_df[["Target"] + [c for c in converter_params_df.columns if c != "Target"]]

In [4]:
def get_train(data_dict, s, rep=None):
    data = data_dict[f"subj{s}"].select("ROI_VC")
    labels = data_dict[f"subj{s}"].select("image_index")
    sort_index = np.argsort(labels.flatten())
    data = data[sort_index, :]
    labels = labels[sort_index]

    if rep is not None:
        mask_unit = np.zeros(5, dtype=bool)
        mask_unit[:rep] = True
        mask = np.tile(mask_unit, 1200)
        data = data[mask]
        labels = labels[mask]

    return data, labels

def get_test(data_dict, s):
    data = data_dict[f"subj{s}"].select("ROI_VC")
    labels = data_dict[f"subj{s}"].select("image_index")
    sort_index = np.argsort(labels.flatten())
    data = data[sort_index, :]
    labels = labels[sort_index]
    
    return data, labels

In [5]:
def calculate_pattern_correlation(target_brain, recon_brain, label, rep=24):
    sort_idx = np.argsort(label.flatten())
    target_brain = target_brain[sort_idx]
    recon_brain = recon_brain[sort_idx]
    label = label[sort_idx]
    unique_label = np.unique(label)

    pattern_corr = []
    for image_idx in unique_label:
        target_pattern = target_brain[(label == image_idx).flatten(), :]
        recon_pattern = recon_brain[(label == image_idx).flatten(), :]
        
        corrs = np.corrcoef(target_pattern, recon_pattern)[:rep, rep:]
        corr = np.mean(corrs) 

        pattern_corr.append(corr)

    return pattern_corr

def calculate_profile_correlation(target_brain, recon_brain, label, rep=24):
    sort_idx = np.argsort(label.flatten())
    target_brain = target_brain[sort_idx]
    recon_brain = recon_brain[sort_idx]

    profile_corr = []
    for voxel_idx in range(target_brain.shape[1]):
        target_profile = target_brain[:, voxel_idx].reshape(rep, -1, order="F")
        recon_profile = recon_brain[:, voxel_idx].reshape(rep, -1, order="F")
        
        corrs = np.corrcoef(target_profile, recon_profile)[:rep, rep:]
        corr = np.mean(corrs) 

        profile_corr.append(corr)

    return profile_corr

def save_result(result_data, output_dir, output_filename):
    """
    Save the results to a CSV file.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    df = pd.DataFrame(result_data)
    df.to_csv(os.path.join(output_dir, output_filename), index=None)

In [6]:
pattern_corr_result = []
profile_corr_result = []

for subj_target, subj_source in permutations(subj_list, 2):
    s_t = f"{int(subj_target):02d}"
    s_s = f"{int(subj_source):02d}"

    # === Train ===
    x_train, x_train_labels = get_train(train_brain_data_dict, s_s, rep)
    y_train, y_train_labels = get_train(train_brain_data_dict, s_t, rep)

    # normalize
    x_train_mean = np.mean(x_train, axis=0)[np.newaxis, :]
    x_train_norm = np.std(x_train, axis=0, ddof=1)[np.newaxis, :]
    x_train_normalized = (x_train - x_train_mean) / x_train_norm
    y_train_mean = np.mean(y_train, axis=0)[np.newaxis, :]
    y_train_norm = np.std(y_train, axis=0, ddof=1)[np.newaxis, :]
    y_train_normalized = (y_train - y_train_mean) / y_train_norm

    # fit
    model = FastL2LiR()
    alpha = converter_params_df.set_index(["Target", "Source", "Number of samples"]).at[(f"{s_t}", f"{s_s}", num_samples), "Alpha"]
    model.fit(x_train_normalized,  y_train_normalized, alpha=780, n_feat=x_train_normalized.shape[1], chunk_size=0, dtype=np.float32)


    # === Test ===
    x_test, x_test_labels = get_test(test_brain_data_dict, s_s)
    y_test, y_test_labels = get_test(test_brain_data_dict, s_t)

    # normalize
    x_test_normalized = (x_test - x_train_mean) / x_train_norm
    y_test_normalized = (y_test - y_train_mean) / y_train_norm

    # predict
    converted_x_test_normalized = model.predict(x_test_normalized)


    # === Evaluation ===
    pattern_corr = calculate_pattern_correlation(y_test_normalized, converted_x_test_normalized, x_test_labels)
    for i, corr in enumerate(pattern_corr):
        pattern_corr_result.append({
            'Subject_target': subj_target, 
            'Subject_source': subj_source,           
            'Correlation': corr, 
            'Image_idx': i+1}
        )

    profile_corr = calculate_profile_correlation(y_test_normalized, converted_x_test_normalized, x_test_labels)
    for i, corr in enumerate(profile_corr):
        profile_corr_result.append({
            'Subject_target': subj_target, 
            'Subject_source': subj_source,           
            'Correlation': corr, 
            'Voxel_idx': i}
        )

save_result(pattern_corr_result, conversion_output_path, "pattern_correlation_converter_regression.csv")
save_result(profile_corr_result, conversion_output_path, "profile_correlation_converter_regression.csv")